In [ ]:
#accuracy is not that great; it could be great if I could train.
# I wanted to train the model bt my pc is not in that state.

## This comment is after submitting the file. I should have use Mediapipe Hands and MediaPipe Pose instead of yolov8n-pose for better accuracy.
## My mistake, tarai vul kore felechi.

!pip install ultralytics opencv-python


import cv2
from ultralytics import YOLO
import numpy as np
from google.colab import files
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
print("Please upload your video file (desk_video.mp4):")
uploaded = files.upload()

# Get the uploaded file name
video_filename = list(uploaded.keys())[0]
print(f"Video uploaded: {video_filename}")

# Rename to expected filename if needed
if video_filename != "desk_video.mp4":
    os.rename(video_filename, "desk_video.mp4")
    print("File renamed to desk_video.mp4")

Please upload your video file (desk_video.mp4):


Saving desk_video.mp4 to desk_video (1).mp4
Video uploaded: desk_video (1).mp4
File renamed to desk_video.mp4


In [ ]:
## checking if the person is raising their hand

def is_hand_raised(keypoints, threshold=50):
    """
    Check if hand is raised based on keypoint positions
    keypoints: flattened array of [x1, y1, conf1, x2, y2, conf2, ...]
    """
    LEFT_SHOULDER = 5
    LEFT_WRIST = 9
    RIGHT_SHOULDER = 6
    RIGHT_WRIST = 10

    def check_hand(shoulder_idx, wrist_idx):
        # Get shoulder coordinates
        sx = keypoints[shoulder_idx * 3]
        sy = keypoints[shoulder_idx * 3 + 1]
        s_conf = keypoints[shoulder_idx * 3 + 2]

        # Get wrist coordinates
        wx = keypoints[wrist_idx * 3]
        wy = keypoints[wrist_idx * 3 + 1]
        w_conf = keypoints[wrist_idx * 3 + 2]

        # Check if keypoints are valid (confidence > 0.5)
        if s_conf > 0.5 and w_conf > 0.5:
            # Hand is raised if wrist is significantly above shoulder
            return wy < (sy - threshold)
        return False

    # Check both left and right hands
    left_raised = check_hand(LEFT_SHOULDER, LEFT_WRIST)
    right_raised = check_hand(RIGHT_SHOULDER, RIGHT_WRIST)

    return left_raised or right_raised

def get_person_center(keypoints):
    """Get the center point of a person based on their keypoints"""
    # Use nose (index 0) as reference point, or shoulder center if nose not available
    nose_x = keypoints[0]
    nose_y = keypoints[1]
    nose_conf = keypoints[2]

    if nose_conf > 0.5:
        return nose_x, nose_y
    else:
        # Use shoulder center as fallback
        left_shoulder_x = keypoints[5 * 3]
        left_shoulder_y = keypoints[5 * 3 + 1]
        right_shoulder_x = keypoints[6 * 3]
        right_shoulder_y = keypoints[6 * 3 + 1]

        center_x = (left_shoulder_x + right_shoulder_x) / 2
        center_y = (left_shoulder_y + right_shoulder_y) / 2
        return center_x, center_y

def get_person_name(person_x, person_y, frame_width, frame_height):

#### mapping the position all though the model couldn't detect all of them
    seating_map = [
        ("TANVIR", 0.0, 0.2, 0.3, 0.7),
        ("SHAHADAT", 0.15, 0.35, 0.3, 0.7),
        ("FAISAL", 0.15, 0.35, 0.7, 1.0),
        ("TOUFIQ", 0.35, 0.55, 0.3, 0.7),
        ("MUFRAD", 0.35, 0.55, 0.3, 0.7),
        ("ANIK", 0.35, 0.55, 0.7, 1.0),
        ("IMRAN", 0.55, 0.75, 0.3, 0.7),
        ("EMON", 0.75, 1.0, 0.3, 0.7),
    ]

    # Convert person position to ratios
    x_ratio = person_x / frame_width
    y_ratio = person_y / frame_height

    # Find matching person
    for name, min_x, max_x, min_y, max_y in seating_map:
        if min_x <= x_ratio <= max_x and min_y <= y_ratio <= max_y:
            return name

    return "UNKNOWN"

print("Functions defined successfully!")

Functions defined successfully!


In [ ]:
# Loading model and processing the frames

model = YOLO("yolov8n-pose.pt")

# Open video file
cap = cv2.VideoCapture("desk_video.mp4")
if not cap.isOpened():
    print("Error accessing the video file.")
    exit()

# Get video properties for output
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video properties: {width}x{height} at {fps} FPS, {total_frames} frames")

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_hand_detection.mp4', fourcc, fps, (width, height))

print("Processing video...")
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 30 == 0:  # Print progress every 30 frames
        print(f"Processing frame {frame_count}/{total_frames}")

    # Run YOLO pose detection
    results = model(frame)

    # Get the annotated frame
    output_frame = results[0].plot()

    raised_hands = []

    # Check each detected person
    if results[0].keypoints is not None:
        for person_keypoints in results[0].keypoints.data:
            # Convert to numpy array and flatten
            keypoints = person_keypoints.cpu().numpy().flatten()

            # Check if this person has raised hand
            if is_hand_raised(keypoints):
                # Get person's position
                person_x, person_y = get_person_center(keypoints)

                # Get person's name based on position
                person_name = get_person_name(person_x, person_y, width, height)

                print(f"{person_name} raised hand!")
                raised_hands.append(person_name)

    # Display names on frame if hands are raised
    if raised_hands:
        names_text = ", ".join(raised_hands)
        cv2.putText(output_frame, names_text, (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

    # Write frame to output video
    out.write(output_frame)

cap.release()
out.release()
print("Video processing completed. Output saved as 'output_hand_detection.mp4'")

Video properties: 1920x1080 at 29 FPS, 1081 frames
Processing video...

0: 384x640 5 persons, 153.7ms
Speed: 4.0ms preprocess, 153.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 142.6ms
Speed: 3.4ms preprocess, 142.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 162.4ms
Speed: 4.1ms preprocess, 162.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 155.4ms
Speed: 4.2ms preprocess, 155.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 159.6ms
Speed: 3.4ms preprocess, 159.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 143.1ms
Speed: 3.4ms preprocess, 143.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 153.4ms
Speed: 3.1ms preprocess, 153.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 160

In [ ]:
if os.path.exists('output_hand_detection.mp4'):
    print("Output video created successfully!")
    print("File size:", os.path.getsize('output_hand_detection.mp4'), "bytes")

    # Download the output video
    files.download('output_hand_detection.mp4')
    print("Download started!")
else:
    print("Error: Output video not found!")

Output video created successfully!
File size: 62413089 bytes


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download started!
